## 네이버 증권뉴스 data 가져오기

### module import

In [1]:
from selenium.webdriver import Chrome

from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from pywinauto.keyboard import send_keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.select import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as expected_conditions

import json

import time, os

### init

In [2]:
# url
url = "https://finance.naver.com/news/mainnews.naver" # naver finance main news

In [3]:
# driver init & 접속
driver = Chrome()
driver.get(url)
# time.sleep(1)

### 아티클 제목만 추출

In [4]:
# 아티클 제목만 추출
subjects = driver.find_elements(By.CSS_SELECTOR, ".articleSubject")
subjects

# 제목 요소의 텍스트 추출
for subject  in subjects:
    print(subject.text)

기업거버넌스포럼 “파행적 고려아연 임시 주총에 국격 추락”
암호화폐 베테랑 "비트코인 2~3월 13만달러 돌파"
삼성운용 'KODEX 200타겟위클리커버드콜', 두 달만에 순자산 1000억 돌파
연휴 후 한국 증시, 반도체주 '딥시크 쇼크'
메모리 양극화 심화에…삼성전자 4분기 반도체 영업익 2.9조(종합)
‘딥시크 쇼크’에 관련주 희비…‘반도체’ 울고 ‘AI소프트웨어’ 웃고
"백종원 믿고 샀는데" 신저가 찍은 더본코리아…99.99%가 '비명'
미래운용 ‘TIGER 미국필라델피아AI반도체나스닥’, 순자산 3000억 돌파
‘반도체 뒷걸음질’ 삼성전자…외국인 보유 비중 2년來 최저 [투자360]
“투자자 99%가 손실이라니”…리니지 제국 다시 부활할까 [K주식 이걸 사? 말아?]
딥시크 충격에 SK하이닉스 약세…네이버·카카오 급등
"언젠간 오르겠지" 버텼는데 적자 확대…호텔신라 목표가도 '뚝뚝'
`딥시크 쇼크` 국장 덮쳤다…SK하이닉스 10%대↓
현금부자 주목, KODEX 200타겟위클리커버드콜…순자산 1000억 돌파
허선호 미래에셋증권 대표 "서학개미 13조 벌어, 중국·인도로 분산투자"
“미·중 AI 패권전쟁 시작됐다”…엔비디아 누른 ‘가성비 AI’ 딥시크 파장 어디까지? [투자360]
"네이버도 할 수 있는 거 아냐?"…'딥시크 등장'에 주가 급등
기다리던 엔비디아 HBM 납품 소식인데…삼성전자, 2%대 하락[핫종목]
'딥시크 쇼크' 20만닉스도 위태…반도체주 줄줄이 '휘청'
앱트뉴로사이언스-에이프로젠, 파킨슨병 치료제 공동개발 계약 체결


### 아티클에서 여러 요소 추출

In [5]:
# 아티클 영역 요소 선택
articles = driver.find_elements(By.CSS_SELECTOR, ".block1")

# 각 아티클 영역 요소에서 세부 요소 추출
article_list = []
for article in articles:
    subject = article.find_element(By.CSS_SELECTOR, ".articleSubject").text
    summary = article.find_element(By.CSS_SELECTOR, ".articleSummary").text
    press   = article.find_element(By.CSS_SELECTOR, ".press").text
    wdate   = article.find_element(By.CSS_SELECTOR, ".wdate").text
    link    = article.find_element(By.CSS_SELECTOR, ".articleSubject>a").get_attribute("href")
    article_list.append([subject, summary, press, wdate, link])


### 파일로 다운로드

In [6]:
import pandas as pd

df = pd.DataFrame(article_list,
                columns=['subject', 'summary', 'press', 'wdate', 'link'])
df.head(3)

,subject,summary,press,wdate,link
0,기업거버넌스포럼 “파행적 고려아연 임시 주총에 국격 추락”,최대주주 영풍의 의결권을 기습적으로 배제하면서 파행적으로 진행된 고려아연의 임시주주...,데일리안,2025-01-31 10:56:13,https://n.news.naver.com/mnews/article/119/000...
1,"암호화폐 베테랑 ""비트코인 2~3월 13만달러 돌파""",월가에서 암호화폐 베테랑으로 불리는 제프 켄드릭 스탠다드차타드(SC) 애널리스트가 ...,한국경제TV,2025-01-31 10:43:15,https://n.news.naver.com/mnews/article/215/000...
2,"삼성운용 'KODEX 200타겟위클리커버드콜', 두 달만에 순자산 1000억 돌파",최근 ‘세금 없는’ 국내 주식 시세차익과 장내 파생상품 수익을 활용해 매월 고배당과...,파이낸셜뉴스,2025-01-31 10:34:36,https://n.news.naver.com/mnews/article/014/000...


In [8]:
df.to_csv('./data/네이버증권뉴스.csv', encoding='utf-8', index=False)

import openpyxl
df.to_excel('./data/네이버증권뉴스.xlsx', index=False)

### 특정 날짜의 기사 수집

In [9]:
'''
url ex) https://finance.naver.com/news/mainnews.naver?date=2025-01-23
'''

'https://finance.naver.com/news/mainnews.naver'

In [12]:
import datetime
date = input("검색할 날짜(yyyymmdd) : ")
if date=="": date = datetime.datetime.now().strftime('%Y-%m-%d')
else: date = f'{date[:4]}-{date[4:6]}-{date[6:]}'
url2 = url + "?date=" + date

driver.get(url2)

In [13]:
# 파일명 관리
file_name = date.replace('-', '')
df.to_csv(f"./data/네이버증권뉴스_{file_name}.csv", encoding='utf-8', index=False)

df.to_excel(f'./data/네이버증권뉴스_{file_name}.xlsx', index=False)

### 여러 페이지 크롤링

In [ ]:
'''
url 
ex) https://finance.naver.com/news/mainnews.naver?date=2025-01-23&page=1
ex) https://finance.naver.com/news/mainnews.naver?date=2025-01-23&page=2
ex) https://finance.naver.com/news/mainnews.naver?date=2025-01-23&page=3
'''

In [14]:
# 모든 페이지 크롤링
article_list = []
page = 1
while True:
    page_url = f'{url2}&page={page}'
    driver.get(page_url)
    time.sleep(2) # 페이지가 로딩되는 시간 감안하여 delay

    # 아티클 영역 요소 선택
    articles = driver.find_elements(By.CSS_SELECTOR, ".block1")

    # 아티클 요소에서 세부 요소 추출
    for article in articles:
        subject = article.find_element(By.CSS_SELECTOR, ".articleSubject").text
        summary = article.find_element(By.CSS_SELECTOR, ".articleSummary").text
        press   = article.find_element(By.CSS_SELECTOR, ".press").text
        wdate   = article.find_element(By.CSS_SELECTOR, ".wdate").text
        link    = article.find_element(By.CSS_SELECTOR, ".articleSubject>a").get_attribute('href')
        article_list.append([subject, summary, press, wdate, link])

    # 다음 페이지 버튼 확인
    try:
        driver.find_element(By.CSS_SELECTOR, ".pgRR")
        page += 1
    except:
        break
driver.quit()

df = pd.DataFrame(article_list,
                columns = ['subject', 'summary', 'press', 'wdate', 'link'])

file_name= date.replace('-', '')
df.to_csv(f'./data/네이버증권뉴스_{file_name}.csv', encoding='utf-8', index=False)
df.to_excel(f'./data/네이버증권뉴스_{file_name}.xlsx', index=False)

### 전체 코드

In [18]:
date = input("검색할 날짜(yyyymmdd) : ")
if date=="": date = datetime.datetime.now().strftime('%Y-%m-%d')
else: date = f'{date[:4]}-{date[4:6]}-{date[6:]}'
url = 'https://finance.naver.com/news/mainnews.naver'
date_url = url + "?date=" + date

driver = Chrome()
driver.get(date_url)

# 모든 페이지 크롤링
article_list = []
page = 1
while True:
    page_url = f'{date_url}&page={page}'
    driver.get(page_url)
    time.sleep(2) # 페이지가 로딩되는 시간 감안하여 delay

    # 아티클 영역 요소 선택
    articles = driver.find_elements(By.CSS_SELECTOR, ".block1")

    # 아티클 요소에서 세부 요소 추출
    for article in articles:
        subject = article.find_element(By.CSS_SELECTOR, ".articleSubject").text
        summary = article.find_element(By.CSS_SELECTOR, ".articleSummary").text
        press   = article.find_element(By.CSS_SELECTOR, ".press").text
        wdate   = article.find_element(By.CSS_SELECTOR, ".wdate").text
        link    = article.find_element(By.CSS_SELECTOR, ".articleSubject>a").get_attribute('href')
        article_list.append([subject, summary, press, wdate, link])

    # 다음 페이지 버튼 확인
    try:
        driver.find_element(By.CSS_SELECTOR, ".pgRR")
        page += 1
    except:
        break
driver.quit()

df = pd.DataFrame(article_list,
                columns = ['subject', 'summary', 'press', 'wdate', 'link'])

file_name= date.replace('-', '')
df.to_csv(f'./data/네이버증권뉴스_{file_name}.csv', encoding='utf-8', index=False)
df.to_excel(f'./data/네이버증권뉴스_{file_name}.xlsx', index=False)

### +) 무한 스크롤

In [ ]:
# 무한스크롤
# 페이지의 스크롤 높이 받아오기
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    # 스크롤 끝까지 내리기
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

    # 현재 스크롤 위치 받아오기
    current_height = driver.execute_script("return document.body.scrollHeight")

    # 마지막스크롤 = 현재스크롤이면 반복 벗어나기  
    if last_height == current_height: break

    # 현재 스크롤을 마지막 스크롤로 업데이트 하기
    last_height = current_height

    time.sleep(1)

### +) 댓글 더보기

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
import time

def load_all_comments(driver, url):
    driver.get(url) # 지정된 URL로 브라우저를 열어 이동
    time.sleep(2) # 페이지 로드 대기

    while True:
        try:
            # "댓글 더보기" 버튼 찾기
            more_comments_button = driver.find_element(By.CSS_SELECTOR, '.u_cbox_in_view_comment')
            more_comments_button.click() # "댓글 더보기" 버튼 클릭
            time.sleep(2) # 댓글 로드 대기
        except (NoSuchElementException, StaleElementReferenceException):
            # "댓글 더보기" 버튼이 더 이상 없거나 요소 참조가 유효하지 않을 경우 반복문 탈출
            break

# 사용 예시
url = "https://n.news.naver.com/mnews/hotissue/article/028/0002728835?type=series&cid=2001999" # 네이버 뉴스 기사 URL
driver = webdriver.Chrome() # Chrome 웹드라이버 객체 생성

load_all_comments(driver, url) # 댓글을 모두 로드하는 함수 호출